# Dynamic Programming

## 5. Longest Palindromic Substring

Consider a matrix `dp` where `dp[i][j]` denotes a bool value of whether substring `s[i:j]` is valid parlindromic or not. And obviously, if `i == j` then the string is valid. So we need to fill in 0s on the diagnal.

We need two global variables to tracka the longest substring we have found, using `_being` and `max_len`.

We apply a **sliding window** idea here. The length of the parlindromic substring can range from 2 `'bb'` for example, to the whole string like `'ababa'`. For each window, we start from the leftmost and roll to the right.

While sliding the window, we only compare if the `s[i] == s[j]`, the two end points and,

- if this is a length-2 window, then this is a valid substring.
- if this is a longer one, then the string is valid only if the narrower substring `s[i+1, j-1]` is also valid. (`'ababa'`) is valid only if (`'bab'`) inside is valid also.

After each step, see if the current valid substring has global `max_len`, if so, track the left index and the length of this substring.

In [86]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        from pprint import pprint
        n = len(s)
        dp = [[0] * n for _ in range(n)]
        
        # the diagonal entries are always 1 
        for i in range(n): dp[i][i] = 1
        
        # global variable to track the longest substring we have found
        _begin = 0
        max_len = 1
        
        for _len in range(2, n + 1):
            for l in range(n):
                r = l + _len - 1
                # the right cannot exceed the length
                if r > n-1:
                    break
                    
                if s[l] != s[r]:
                    dp[l][r] = 0
                else:
                    if _len == 2:
                        dp[l][r] = 1
                    else:
                        dp[l][r] = dp[l + 1][r - 1]
                        
                if dp[l][r] == 1 and r - l + 1 > max_len:
                    max_len = r - l + 1
                    _begin = l
                    
                # pprint("Checking : {0} and {1}".format(l+1,r+1))
                # pprint(dp)
                # print("---------")
        return s[_begin:_begin + max_len]

## 647. Palindromic Substrings

In [12]:
class Solution:
    def countSubstrings(self, s: str) -> int:
        # from pprint import pprint
        
        n = len(s)
        dp = [[False] * n for _ in range(n)]
        # a single char is valid
        for i in range(n): dp[i][i] = True
        count = 0
        
        for _len in range(2, n+1):
            for left in range(n):
                right = left + _len - 1
                # boundary condition
                if right > n-1:
                    break
                
                if s[left] == s[right]:
                    if _len == 2:
                        dp[left][right] = True
                    else:
                        dp[left][right] = dp[left+1][right-1]
                else:
                    dp[left][right] = False
                
                if dp[left][right]:
                    count+=1
        
        # pprint(dp)
        return count + n

## 1143. Longest Common Subsequence

https://leetcode.com/problems/longest-common-subsequence/

In [21]:
class Solution:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        n_row = len(text1)
        n_col = len(text2)
        # create the dp matrix and padding the first col and row to be 0
        dp = [[0] * (n_col+1) for _ in range(n_row+1)]
        
        for i in range(1, n_row+1):
            for j in range(1, n_col+1):
                if text1[i-1] == text2[j-1]:
                    dp[i][j] = dp[i-1][j-1] + 1
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i][j-1])
        return dp[-1][-1]

## 64. Minimum Path Sum

In [5]:
class Solution:
    def minPathSum(self, grid: list[list[int]]) -> int:
        n_row = len(grid)
        n_col = len(grid[0])
        
        # build the matrix for dp
        dpm = [[0] * n_col for _ in range(n_row)]
        dpm[0][0] = grid[0][0]
        
        # fill in the first column
        for i in range(1, n_row):
            dpm[i][0] = dpm[i-1][0] + grid[i][0]
        
        for j in range(1, n_col):
            dpm[0][j] = dpm[0][j-1] + grid[0][j]
            
        for j in range(1, n_col):
            for i in range(1, n_row):
                dpm[i][j] = min(dpm[i-1][j] + grid[i][j], dpm[i][j-1] + grid[i][j])
        
        return dpm[-1][-1]

## 152. Maximum Product Subarray

Name the list `nums` as `a`, and use a dynamic programming approach.

Denote $f_{\max}(i)$ be the maximum product of any possible subarrays that end with `a[i]`.

It is not difficult to get: $f_{\max }(i)=\max _{i=1}^{n}\left\{f(i-1) \times a_{i}, a_{i}\right\}$. We either keep `a[i]` within the last sequence, or start a new sequence starting from `a[i]`, which depends on which one is larger.

**Tricky part**: the above is ok if we are deriving the `sum` instead of the `product`. Why? Because there might be negative values in the middle! Before we meet the second negative value to turn the product positive, we might lose track of the sequence product already.

**Solution**: maintain two sequence for memorization. The *max* sequence is waiting for a large positive number, while the *min* sequence is waiting for a large negative number, so we won't lose track of any sequence that has a potential to be the maximum.

\begin{aligned}
f_{\max }(i) &=\max _{i=1}^{n}\left\{f_{\max }(i-1) \times a_{i}, f_{\min }(i-1) \times a_{i}, a_{i}\right\} \\
f_{\min }(i) &=\min _{i=1}\left\{f_{\max }(i-1) \times a_{i}, f_{\min }(i-1) \times a_{i}, a_{i}\right\}
\end{aligned}

In [6]:
class Solution:
    def maxProduct(self, nums: list[int]) -> int:
        n = len(nums)
        dp_max = [nums[0]] * n
        dp_min = [nums[0]] * n
        for i in range(1, n):
            dp_max[i] = max(dp_max[i-1] * nums[i], dp_min[i-1] * nums[i], nums[i])
            dp_min[i] = min(dp_max[i-1] * nums[i], dp_min[i-1] * nums[i], nums[i])
        return max(dp_max)
        

## 198. House Robber

The robber has to choose whether to rob the odd subsequence or the even subsequence. Define the sequence `dp` as the maximum amount of money the robber can get after considerting the house `i`. Then it is not difficult to have the transition:

$d p[i]=\max (d p[i-2]+n u m s[i], d p[i-1])$

In [10]:
class Solution:
    def rob(self, nums: list[int]) -> int:
        n = len(nums)
        dp = [0] * n
        if n == 0:
            return 0
        elif n == 1:
            return nums[0]
        elif n == 2:
            return max(nums[0], nums[1])
        else:
            dp[0] = nums[0]
            dp[1] = max(nums[0], nums[1])
            for i in range(2, n):
                dp[i] = max(dp[i-2] + nums[i], dp[i-1])
        return max(dp)

## 221. Maximal Square

Problem: Given an m x n binary matrix filled with 0's and 1's, find the largest square containing only 1's and return its area.

Denote `dp[i][j]` to be the maximum length of the square that the `i, j` block can attain. Of cource, `matrix[i][j]` has to be "1" at least, and then most importantly, the transition equation is:

`dp[i][j] = min(dp[i-1][j], dp[i-1][j-1], dp[i][j-1]) + 1`

The intuition here is that, to make a square, the current block depends on blocks to its left, top, and top left corner. Only the least of their values are attainable for the group of four blocks (similar to bucket effect)

In [24]:
class Solution:
    def maximalSquare(self, matrix: list[list[str]]) -> int:
        m, n = len(matrix), len(matrix[0])
        dp = [[0] * n for _ in range(m)]
        maxSide = 0
        for i in range(m):
            for j in range(n):
                if matrix[i][j] == "1":
                    # if these blocks on the top, left boundary, keep the original values
                    if i == 0 or j == 0:
                        dp[i][j] = 1
                    else:
                        dp[i][j] = min(dp[i-1][j], dp[i-1][j-1], dp[i][j-1]) + 1
                maxSide = max(maxSide, dp[i][j])
        return maxSide**2

## 279. Perfect Squares

Use a dynamic programming approach, though it is not the optimal. We iterate the sequence from 1 to n, and for each number, our algo is going to find the minmimum number of perfect squares, since we have found the minum number of previous ones, we can utilize the results from the `dp`. There is a transition formula

$f[i]=1+\min _{j=1}^{\lfloor\sqrt{i}\rfloor} f\left[i-j^{2}\right]$

In [1]:
class Solution:
    def numSquares(self, n: int) -> int:
        dp = [0] * (n + 1)
        for i in range(1, n+1):
            minn = 10**4
            for j in range(1, int(i**0.5) + 1):
                minn = min(minn, dp[i - j * j])
            dp[i] = minn + 1
        return dp[n]

## 309. Best Time to Buy and Sell Stock with Cooldown

Define three states of the `i`th day EOD status.

0. Own a stock
1. Do not own a stock and is in cool down
2. Do not own a stock and is not in cool down

**Possibilities of state 0**:

You either buy a stock today `f[i-1][2] - prices[i]` (then you are do not own it yesterday and you are not in cool down, buy a stock will cost you `prices[i]`), or you keep a stock from yesterday `f[i-1][0]`.

**Possibilities of state 1**:

You sell the stock today, which means you had a stock yesterday. `f[i-1][0] + prices[i]`.

**Possibilities of state 2**:

You do not own the stock and you didn't sell it today. Then you either not in cool down yesterday, or you sold the stock yesterday and the cool down has passed. `max(f[i-1][1], f[i-1][2])`

**Note**:

On day 0, there is no cool down, so all the max profits for states 1/2 are 0, and for state 0 is `-prices[0]` if we hypothetically buy.

In [8]:
class Solution:
    def maxProfit(self, prices: list[int]) -> int:
        n = len(prices)
        dp = [[0] * 3 for _ in range(n)]
        for i in range(n):
            if i == 0:
                dp[i][0] = -prices[0]
                dp[i][1] = 0
                dp[i][2] = 0
            else:
                dp[i][0] = max(dp[i - 1][0], dp[i - 1][2] - prices[i])
                dp[i][1] = dp[i - 1][0] + prices[i]
                dp[i][2] = max(dp[i - 1][1], dp[i - 1][2])
        return max(dp[-1][0], dp[-1][1], dp[-1][2])

## 0-1 Knap Sack Problem

**Problem** Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.

**Solution** Define the matrix `dp`, with `dp[i][j]` indicating the maximum value can be attained using the first `i` items with the constraint weight of `j`.

To facilitate the dynamic programming, note that we create a matrix with `n_rows = n_items + 1`, and `n_cols = total_wgt + 1`, since the first column and first row will always be zero. Zero items or zero weights should have no value at all, so we leave them as 0s.

Then for the rest of the loop, we essentially just do two things:

1. If the current item weight bigger than the constaint? If yes, then we omit this item, just fill in the current max value.
2. If the current item weight smaller. Then, we either do nothing, or include this adding with the maximum value under the new constraints. The relationship is therefore:

`dp[i][j] = max(dp[i-1][j], item_vals[i-1] + dp[i-1][j-item_wgts[i-1]])`

In [ ]:
def knapsack01(item_wgts, item_vals, tot_wgt):
    from pprint import pprint
    n = len(item_wgts)
    dp = [[0]*(tot_wgt + 1) for _ in range(n + 1)]
    for i in range(1, n + 1):
        for j in range(1, tot_wgt + 1):
            if item_wgts[i - 1] > j:
                dp[i][j] = dp[i-1][j]
            else:
                dp[i][j] = max(dp[i-1][j], item_vals[i-1] + dp[i-1][j-item_wgts[i-1]])
                
    pprint(dp)
    return dp[-1][-1]

# item_weights = [2, 3, 5, 1, 12]
# item_values = [11, 9, 3, 7, 18]
# total_weight = 10
# knapsack01(item_weights, item_values, total_weight)

## 416. Partition Equal Subset Sum

**Problem:** Given a non-empty array nums containing only positive integers, find if the array can be partitioned into two subsets such that the sum of elements in both subsets is equal.

**Solution:** The only difference here is that instead of intializing 0s, `False` is entered. And we need a separate statement `nums[i-1] == j` to offer some true values so that the recursion is valid. (or all the values will be `False`)

In [40]:
class Solution:
    def canPartition(self, nums: list[int]) -> bool:
        from pprint import pprint
        s = sum(nums)
        if s % 2 == 1:
            return False
        s = int(s / 2)
        n = len(nums)
        dp = [[False] * (s+1) for _ in range(n+1)]
        for i in range(1, n+1):
            for j in range(1, s+1):
                if nums[i-1] == j:
                    dp[i][j] = True
                elif nums[i-1] > j:
                    dp[i][j] = dp[i-1][j]
                else:
                    dp[i][j] = max(dp[i-1][j], dp[i-1][j-nums[i-1]])
        return dp[-1][-1]

## 494. Target Sum

**Problem:** You are given an integer array nums and an integer target.

You want to build an expression out of nums by adding one of the symbols '+' and '-' before each integer in nums and then concatenate all the integers.

For example, if nums = [2, 1], you can add a '+' before 2 and a '-' before 1 and concatenate them to build the expression "+2-1".
Return the number of different expressions that you can build, which evaluates to target.

In [44]:
class Solution:
    def findTargetSumWays(self, nums: list[int], target: int) -> int:
        pass

## 1049. Last Stone Weight II

**Problem:** You are given an array of integers stones where stones[i] is the weight of the ith stone.

We are playing a game with the stones. On each turn, we choose any two stones and smash them together. Suppose the stones have weights x and y with x <= y. The result of this smash is:

If x == y, both stones are destroyed, and
If x != y, the stone of weight x is destroyed, and the stone of weight y has new weight y - x.
At the end of the game, there is at most one stone left.

Return the smallest possible weight of the left stone. If there are no stones left, return 0

**Solution:** This is again a variation of the 0-1 Knap Sack problem. Given a constraint and a collection of items, we want to find out the maximum value can be attained by combinations of the items.

Here, we want to find a combination of items that are close to `(sum/2)`.


In [10]:
class Solution:
    def lastStoneWeightII(self, stones: list[int]) -> int:
        s = sum(stones) // 2
        n = len(stones)
        dp = [[0] * (s+1) for _ in range(n+1)]
        for i in range(1, n+1):
            for j in range(1, s+1):
                if stones[i-1] > j:
                    dp[i][j] = dp[i-1][j]
                else:
                    dp[i][j] = max(dp[i-1][j], stones[i-1] + dp[i-1][j-stones[i-1]])
        
        return sum(stones) - 2*dp[-1][-1]

## 322. Coin Change

**Problem:** You are given an integer array coins representing coins of different denominations and an integer amount representing a total amount of money.

Return the fewest number of coins that you need to make up that amount. If that amount of money cannot be made up by any combination of the coins, return -1.

You may assume that you have an infinite number of each kind of coin.

**Solution:** Define a sequence `dp`, with `dp[j]` indicating the minimum number of coins to attain value `j`.

**Note:** When initializing the `dp` sequence, intilize as `amount + 1`, which is a "large" value that should be replaced by any possible coin combinations since the minimum face value is 1. Also, at the end of the return function, we can easily identify if this value is attainable by the coins or not, so we can wisely return -1.

In [86]:
class Solution:
    def coinChange(self, coins: list[int], amount: int) -> int:
        # from pprint import pprint
        dp = [amount + 1] * (amount + 1)
        dp[0] = 0
        for coin in coins:
            for j in range(coin, amount + 1):
                dp[j] = min(dp[j], dp[j - coin] + 1)
                # pprint(dp)
        return -1 if dp[-1] > amount else dp[-1]

##  377. Combination Sum IV

**Problem:** Given an array of distinct integers nums and a target integer target, return the number of possible combinations that add up to target.

The test cases are generated so that the answer can fit in a 32-bit integer.

**Solution:** 

- `dp` definition: Define `dp[i]` as number of combinations available to get target `i`.
- Boundary condition: When `i=0`, then only one combination, an empty set can satisfy, the boundary condition `dp[0] = 0`.
- Transition: for each number in `nums`, `dp[i] += dp[i - num]`

In [5]:
class Solution:
    def combinationSum4(self, nums: list, target: int) -> int:
        n = len(nums)
        dp = [0] * (target + 1)
        dp[0] = 1
        for i in range(1, target + 1):
            for num in nums:
                if num <= i:
                    dp[i] += dp[i - num]
        return dp[-1] 

## 2218. Maximum Value of K Coins From Piles

In [7]:
class Solution:
    def maxValueOfCoins(self, piles: list[list[int]], k: int) -> int:
        pass